#visium processing

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)

In [ ]:
packageVersion("Seurat") 

In [ ]:
visium_dir <- file.path("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/HN00218008_auto_result_10X/S24-6440/")

image_dir <- file.path(visium_dir, "binned_outputs/square_008um/spatial/")
image_hires <- Read10X_Image(
    image_dir,
    image.name = "tissue_hires_image.png",
   
    assay = "Spatial.008um", slice="slice1.008", filter.matrix=TRUE
)

In [ ]:
image_hires

In [ ]:
sample_dir <- "/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/HN00218008_auto_result_10X/S24-6440/"
object <- Load10X_Spatial(
  data.dir = sample_dir,  bin.size  = 8, 
  image    = image_hires  
)

In [ ]:
object

In [ ]:
DefaultAssay(object) <- "Spatial.008um"

In [ ]:
#manually selected tissue area barcodes
library(readr)

barcode_data <-read_csv("new.csv")
barcode_data

In [ ]:
barcodes <- barcode_data[[1]]

In [ ]:
subset <- subset(object, cells=barcodes)
subset

In [ ]:
object <- subset

In [ ]:
vln.plot <- VlnPlot(subset, features = "nCount_Spatial.008um", pt.size = 0) + theme(axis.text = element_text(size = 4)) + NoLegend()
count.plot <- SpatialFeaturePlot(subset, features = "nCount_Spatial.008um", pt.size.factor = 3) + theme(legend.position = "right") 

In [ ]:
object[["percent.mt"]] <- PercentageFeatureSet(object, pattern = "^MT-")

In [ ]:
# Visualize QC metrics as a violin plot
VlnPlot(object, features = c("nFeature_Spatial.008um", "nCount_Spatial.008um", "percent.mt"), ncol = 3)

In [ ]:
object <- subset(object, nCount_Spatial.008um > 35 & nFeature_Spatial.008um > 10 &percent.mt < 10)

In [ ]:
object

In [ ]:
DefaultAssay(object) <- "Spatial.008um"

In [ ]:
s1 <- object

In [ ]:
p<-SpatialFeaturePlot(s1, features = "nCount_Spatial.008um", pt.size.factor = 3, image.scale='hires',  image.alpha=0.4 )

In [ ]:
p

In [ ]:
visium_dir <- file.path("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/HN00218008_auto_result_10X/S21-31525/")

image_dir <- file.path(visium_dir, "binned_outputs/square_008um/spatial/")
image_hires <- Read10X_Image(
    image_dir,
    image.name = "tissue_hires_image.png",
    
    assay = "Spatial.008um", slice="slice1.008", filter.matrix=TRUE
)

In [ ]:
image_hires

In [ ]:
sample_dir <- "/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/HN00218008_auto_result_10X/S21-31525/"
object <- Load10X_Spatial(
  data.dir = sample_dir,  bin.size  = 8, 
  image    = image_hires  
)

In [ ]:
#manually selected tissue area barcodes

barcode_data <-read_csv("new3.csv")
barcode_data

In [ ]:
object@meta.data

In [ ]:
barcodes <- barcode_data[[1]]

In [ ]:
subset <- subset(object, cells=barcodes)
subset

In [ ]:
object <- subset

In [ ]:
colnames(object) <- paste0(colnames(object), "-2") #add '-2' before merging to differentiate tissue 1 and tissue 2

In [ ]:
object@meta.data

In [ ]:
vln.plot <- VlnPlot(subset, features = "nCount_Spatial.008um", pt.size = 0) + theme(axis.text = element_text(size = 4)) + NoLegend()
count.plot <- SpatialFeaturePlot(subset, features = "nCount_Spatial.008um", pt.size.factor = 3) + theme(legend.position = "right") 
vln.plot | count.plot

In [ ]:
object[["percent.mt"]] <- PercentageFeatureSet(object, pattern = "^MT-")

In [ ]:
VlnPlot(object, features = c("nFeature_Spatial.008um", "nCount_Spatial.008um", "percent.mt"), ncol = 3, )

In [ ]:
object <- subset(object, nCount_Spatial.008um > 35 & nFeature_Spatial.008um > 10 &percent.mt < 10)

In [ ]:
length(object$nCount_Spatial.008um)

In [ ]:
DefaultAssay(object) <- "Spatial.008um"

In [ ]:
s2 <- object

In [ ]:
p<-SpatialFeaturePlot(s2, features = "nCount_Spatial.008um", pt.size.factor = 3, image.scale='hires',  image.alpha=0.4 )

In [ ]:
p

In [ ]:
s1
s2

In [ ]:
s1@meta.data['slide'] = 's1'
s2@meta.data['slide'] = 's2'

In [ ]:
s1@images

In [ ]:
s2@images

In [ ]:
names(s1@images)[names(s1@images) == "slice1.008um"] <- "slide1"
names(s2@images)[names(s2@images) == "slice1.008um"] <- "slide2"

In [ ]:
s1
s2

In [ ]:
seurat_obj= s1
seurat_obj$barcode <- colnames(seurat_obj)

tissue_positions = Read10X_Coordinates("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/HN00218008_auto_result_10X/S24-6440/binned_outputs/square_008um/spatial/tissue_positions.parquet", T)
tissue_positions$barcode = rownames(tissue_positions)
tissue_positions <- subset(tissue_positions, barcode %in% seurat_obj$barcode)

new_meta <- dplyr::left_join(seurat_obj@meta.data, tissue_positions, by='barcode')
rownames(new_meta) = rownames(seurat_obj@meta.data)
seurat_obj@meta.data = new_meta
s1 = seurat_obj

In [ ]:
rownames(tissue_positions)

In [ ]:
seurat_obj= s2
seurat_obj$barcode <- colnames(seurat_obj)

In [ ]:
seurat_obj$barcode

In [ ]:
tissue_positions = Read10X_Coordinates("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/HN00218008_auto_result_10X/S21-31525/binned_outputs/square_008um/spatial/tissue_positions.parquet", T)
tissue_positions$barcode = paste0(rownames(tissue_positions), "-2")

In [ ]:
tissue_positions$barcode

In [ ]:
seurat_obj$barcode

In [ ]:
tissue_positions

In [ ]:
rownames(tissue_positions) <- paste0(rownames(tissue_positions), "-2")

In [ ]:
tissue_positions

In [ ]:
tissue_positions <- subset(tissue_positions, barcode %in% seurat_obj$barcode)

new_meta <- dplyr::left_join(seurat_obj@meta.data, tissue_positions, by = 'barcode')
rownames(new_meta) = rownames(seurat_obj@meta.data)
seurat_obj@meta.data = new_meta
s2 = seurat_obj

In [ ]:
s2$slide %>% head()

In [ ]:
object <- merge(s1, s2)

In [ ]:
object <- object %>%
  NormalizeData() %>%
  FindVariableFeatures() %>%
  ScaleData() %>%
  RunPCA()

In [ ]:
object

In [ ]:
DefaultAssay(object)

In [ ]:
object <- FindNeighbors(object, dims = 1:30)
object <- FindClusters(object, verbose = FALSE, resolution =0.5)
object <- RunUMAP(object, dims = 1:30)

In [ ]:
# set factor level for anterior / posterior
object$slide <- factor(as.character(object$slide), levels=c('slide1', 'slide2'))

# show the UMAP
p1 <- DimPlot(object, label=TRUE, reduction = "umap") + NoLegend()
p1

In [ ]:
object <- FindClusters(object, verbose = FALSE, resolution =0.5, cluster.name="cluster")

In [ ]:
SpatialDimPlot(object, label = T, repel = T, label.size = 4, image.scale='hires',  image.alpha=0.4)

In [ ]:
object$label <- "Unknown"

In [ ]:
object$label[WhichCells(object, idents = c(0))] <- "Fibroblast"
object$label[WhichCells(object, idents = c(1))] <- "Basal KC"
object$label[WhichCells(object, idents = c(2))] <- "Lymphoid-FRC niche"
object$label[WhichCells(object, idents = c(3))] <- "Endo/Peri"
object$label[WhichCells(object, idents = c(4))] <- "Differentiating KC"
object$label[WhichCells(object, idents = c(5))] <- "Inflammatory Fib/Mac"
object$label[WhichCells(object, idents = c(6))] <- "Neutrophilic pustule"
object$label[WhichCells(object, idents = c(7))] <- "Pustular KC"
object$label[WhichCells(object, idents = c(8))] <- "a/mDC"
object$label[WhichCells(object, idents = c(9))] <- "pDC" 
object$label[WhichCells(object, idents = c(10))] <- "Terminally diff KC"
object$label[WhichCells(object, idents = c(11))] <- "Mast cell"
object$label[WhichCells(object, idents = c(12))] <- "Pustular KC"
object$label[WhichCells(object, idents = c(13))] <- "Adipocyte"  #adipoq
object$label[WhichCells(object, idents = c(14))] <- "Eccrine gland duct" 
object$label[WhichCells(object, idents = c(15))] <- "Eccrine gland coil" 
object$label[WhichCells(object, idents = c(16))] <- "Terminally diff KC" 
object$label[WhichCells(object, idents = c(17))] <- "Terminally diff KC" 
object$label[WhichCells(object, idents = c(18))] <- "Neutrophilic pustule" 
object$label[WhichCells(object, idents = c(19))] <- "Neutrophilic pustule" 

In [ ]:
Idents(object) <- "label"

In [ ]:
DimPlot(object, label = T) 

In [ ]:
saveRDS(object, "./RDS/combined_3.3_hires.rds")

In [ ]:
object_subset <- subset(object, cells = Cells(object[["Spatial.008um"]]), downsample = 10000)

In [ ]:
object_subset <- JoinLayers(object_subset)

In [ ]:
markers <- FindAllMarkers(object_subset, assay = "Spatial.008um", only.pos = TRUE)
markers %>%
  group_by(cluster) %>%
  dplyr::filter(avg_log2FC > 1) %>%
  slice_head(n = 10) %>%
  ungroup() -> top10

In [ ]:
top10

In [ ]:
write.csv(top10, file="top10.cluster.csv", row.names=FALSE)

In [ ]:
markers %>%
  group_by(cluster) %>%
  dplyr::filter(avg_log2FC > 1) %>%
  slice_head(n = 20) %>%
  ungroup() -> top20

In [ ]:
write.csv(top20, file="top20.cluster.csv", row.names=FALSE)